# <center>Livrable Projet DATA SCIENCE</center>

!pip3 install opencv-python

## Livrable 1 - Prétraitement (denoising/sharpening…)

Le but est de traiter un ensemble de photographies afin de les rendre mieux traitables par les algorithmes de Machine Learning. Il y a deux traitements à réaliser : le débruitage, et l’affutage. Vous devrez produire un notebook Jupyter explicitant ces étapes de prétraitement, et leurs performances. Ces algorithmes s’appuieront sur des notions assez simples autour des filtres de convolution, et les appliqueront pour améliorer la qualité de l’image. Il faudra notamment décider d’un compromis entre dé-bruitage et affutage.

Le notebook devra intégrer :
<ul>
    <li>Le code de chargement du fichier.</li>
    <li>Le code du débruitage sur un sous-ensemble d’images bruitées. Le code doit être accompagné d’explications.</li>
    <li>Le code de l’affutage sur un sous-ensembles d’images floutées. Le code doit être accompagné d’explications.</li>
    <li>
        Une étude de cas explicitant les compromis entre ces deux opérations. Cette partie du livrable doit inclure le bruitage d’images et montrer la perte de détails, ou l’affutage d’images et montrer l’apparition du bruit.
    </li>
</ul>

<b>Ce livrable est à fournir pour le 18/12/2020</b>

In [ ]:
!pip3 install opencv-python

In [ ]:
import os
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import cv2
import threading
from queue import Queue
from multiprocessing import Pool
import time

In [ ]:
def get_image(path, filename):
    return io.imread(path + filename)

def save_image(path, filename, content):
    #Check if folder exists
    if not os.path.isdir(path):
        os.makedirs(path)
    imageio.imwrite(path + filename , content)
    
def get_file_amount(path):
    return len(os.listdir(path))

In [ ]:
def display_image(index_img, image):
    ax = axs[index_img//10, index_img%10]
    ax.axis('off')
    ax.set_title("Image n° "+ str(index_img+1))
    ax.imshow(image)

## Défloutage de l'image

Pour le défloutage des images, on utilise un filtre Laplacien. Ce filtre nous permet d'affuter les images grâce à une fonction de convolution de la librairie opencv sur l'image récupérée.

La variante de filtre choisie nous permet sur le jeu de données fourni d'affuter les images suffisemment pour retirer le flou présent sans pour autant y ajouter de bruit.

In [ ]:
# Deblurring function
def remove_blur(img):
    # Creation of a Laplacian kernel to use for debluring
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    
    # Convolution of the kernel with the image given in the function's parameter
    return cv2.filter2D(img, -1, kernel)

def get_blurry_indicator(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return fm

In [ ]:
# Creer la liste de fichier a traiter
folder = "./Dataset/Blurry/"
listing = os.listdir(folder)

In [ ]:
# Thread execution
def process_fpath(name):
    path = folder+name
    img = get_image(folder,name)

     # Get initial Blur metric
    original_blur_metric = get_blurry_indicator(img)
    
    # Remove blur from the colored image image
    deblurred_img = remove_blur(img)

    # Get initial Blur metric
    processed_blur_metric = get_blurry_indicator(deblurred_img)
    
    print("image N° " + name + " - initial : " + str(original_blur_metric)
        + " - processed : " + str(processed_blur_metric)
        + " - difference : " + str(processed_blur_metric - original_blur_metric)+"\n")
    
    # Saving Image
    save_image("./Dataset/processed/deblurred/", name, deblurred_img)

# boucle sur la liste de fichier
if __name__ == '__main__':
    for name in listing:
        #process_fpath(name)
        t = threading.Thread(target=process_fpath, args=(name,))
        t.start()

## Débruitage
La capture d'un signal lumineux par un appareil photographique s'accompagne le plus souvent d'informations non désirées : le « bruit ». L'essentiel de ce « bruit » (des pixels trop clairs ou trop sombre en trop grand nombre ou de manière irrégulière, par exemple) est dû au capteur.

### Le débruitage par morceaux (par patchs)
Le débruitage par morceaux est une technique de débruitage d'image utilisant l'algorithme de réduction du bruit numérique appelé en Anglais "non-local means".

La méthode repose sur un principe simple, remplacer la couleur d'un pixel par une moyenne des couleurs de pixels similaires. Mais les pixels les plus similaires à un pixel donné n'ont aucune raison d'être proches. Il est donc nécessaire de scanner une vaste partie de l'image à la recherche de tous les pixels qui ressemblent vraiment au pixel que l'on veut débruiter.

### Pourquoi cette methode ?
Le résultat d'un tel filtrage permet d’amoindrir la perte de détails au sein de l'image, comparé aux filtres réalisant des moyennes localement tel que le filtre de Gauss ou le filtre de Wiener, le bruit généré par l'algorithme "non-local means" est plus proche du bruit blanc.

**Syntax:**
cv2.fastNlMeansDenoisingColored( P1, P2, float P3, float P4, int P5, int P6)

**Parameters:**
* P1 – Source Image Array
* P2 – Destination Image Array
* P3 – Size in pixels of the template patch that is used to compute weights.
* P4 – Size in pixels of the window that is used to compute a weighted average for the given pixel.
* P5 – Parameter regulating filter strength for luminance component.
* P6 – Same as above but for color components // Not used in a grayscale image.


In [ ]:
# Remove Noise function
def remove_noise(image):
    return cv2.fastNlMeansDenoisingColored(image,None,10,10,7,15)

In [ ]:
# Prend les nom de tout les fhier dans un dossier donner
folder = "./Dataset/Noisy/"
listing = os.listdir(folder)


In [ ]:
# Thread execution
def process_fpath(name):
    print(name)
    path = folder+name
    img = get_image(folder,name)
    denoised_img = remove_noise(img)
    save_image("./Dataset/processed/test/", name, denoised_img)

#boucle sur la liste de fichier
if __name__ == '__main__':
  for name in listing:
        #process_fpath(name)
        t = threading.Thread(target=process_fpath, args=(name,))
        t.start()
        

# Sources

## Image


## Défloutage


## Debruitage
* https://docs.opencv.org/3.4/d1/d79/group__photo__denoise.html#ga03aa4189fc3e31dafd638d90de335617
* https://docs.opencv.org/3.4/d5/d69/tutorial_py_non_local_means.html
* http://www.ipol.im/pub/art/2011/bcm_nlm/article.pdf
* https://github.com/opencv/opencv/blob/master/modules/photo/src/denoising.cpp
